In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.2f}'.format

## Carga CFI en Mercado COLOCA para Calculo Comisiones
### 1.- Query saldos de custodias:  BCS Server AI
USE BCS
GO

select cod_per Periodo,b.rut_cli Rut,b.sec_rut_cli Secuencia,b.nombre_cli Cliente,b.cod_agente,a.nemo Nemo,sum(CAN_ACTUAL + cAN_GARANTIA) AS CANTIDAD_CUOTAS
 from TBCTCIRV a 
inner join TBPLFICL b on a.rut_cli=b.rut_cli and a.sec_rut_cli=b.sec_rut_cli where cod_per='2023-06' and a.mercado='CF' and COD_BOVEDA='COR' group by cod_per,a.nemo,b.rut_cli,b.sec_rut_cli,b.nombre_cli,b.cod_agente
## 1.- ejecutar query de saldo de custodia al mes anterior
### 2.- Crear Archivo de carga,  con dos pestañas segun formato anterior
### 3.- Revisar Nemos sin Custodia 
### 4.- actualizar fecha periodo para archivo de carga


In [3]:

#archivo= 'Cierre Mensual CFI ENE23.xlsx'
archivo= 'Copia de Cierre Mensual CFI cargamaezo25LAST.xlsx'
patrimonio= pd.read_excel(archivo, sheet_name='custodia')
Facturacion= pd.read_excel(archivo, sheet_name='Consolidado2', header=0).dropna()#, index_col=None, header=None)

#Facturacion.where(Facturacion[1]>0)
#Facturacion.query('Fact>0')
patrimonio['Nemo']=patrimonio['Nemo'].str.strip()
patrimonio.tail(10)

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS
9730,2025-02,78437170-0,0,INVERSIONES PROYECTA LTDA. ...,1000,FIPNPMAT,9217.00
9731,2025-02,78456300-6,0,ASESORIAS E INVERSIONES AIC LIMITADA ...,1000,FIPNPMAT,4615.00
9732,2025-02,78841570-2,0,INVERSIONES CARMEL LTDA ...,3314,FIPNPMAT,10357.00
9733,2025-02,78956200-8,0,COMPAÑIAS DE RENTAS F Y R LTDA. ...,3308,FIPNPMAT,10071.00
9734,2025-02,78989050-1,0,INVERSIONES SAN JUAN MARIA LIMITADA ...,1030,FIPNPMAT,2061.00
9735,2025-02,78989860-K,0,INVERSIONES B Y S LTDA. ...,7200,FIPNPMAT,10325.00
9736,2025-02,79685650-5,0,INVERSIONES LA PINTA LTDA. ...,65,FIPNPMAT,18701.00
9737,2025-02,89293500-9,0,INVERSIONES CARLOMAGNO S.A. ...,1710,FIPNPMAT,2000.00
9738,2025-02,96586750-3,0,NEVASA S.A. CORREDORES DE BOLSA ...,100,FIPNPMAT,51.00
9739,2025-02,96704770-8,0,VAYAL INVERSIONES S.A. ...,1000,FIPNPMAT,6237.00


In [4]:
set(Facturacion.emisor.to_list())



{'BTG', 'Compass'}

In [5]:
patrimonio= patrimonio.merge(Facturacion, left_on='Nemo', right_on='Nemo')#.to_excel('NuevaCargaFip.xlsx')

#Eliminar comisiones A fondos HMC
HMC= {'RUBIX': [9500], 
        'HMC':[7000],
        'ALTAVISTA': [7200], 'ALTAVISTA2': [7201],'B&A GESTION DE ACTIVOS': [9600]}
#Eliminar comisiones A fondos Sartor
Sartor ={'RUBIX': [9500],
       'BD INVESTMENTS': [7696],'BD INVESTMENTS2': [7697],
        'ASESORÍAS E INVERSIONES C&C SPA':[7699]}


ds=pd.DataFrame(Sartor).T
ds2= pd.DataFrame(HMC).T



In [6]:
ds

,0
RUBIX,9500
BD INVESTMENTS,7696
BD INVESTMENTS2,7697
ASESORÍAS E INVERSIONES C&C SPA,7699


In [7]:
patrimonio

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
0,2025-02,5199497-3,0,TAVERNE DENNISON ANABELLA EILY ...,7200,CFIBCCHIGA,102.00,971.00,BTG
1,2025-02,7586678-K,0,ESCOBAR HENRIQUEZ RAUL GUILLERMO...,3314,CFIBCCHIGA,129.00,971.00,BTG
2,2025-02,11347653-2,0,VELASCO PALMA FRANCISCA ...,7200,CFIBCCHIGA,930.00,971.00,BTG
3,2025-02,12752532-3,2,INVERNIZZI CASTILLO XIMENA BEATRIZ...,1638,CFIBCCHIGA,8297.00,971.00,BTG
4,2025-02,13990767-1,0,CABALLERO VEAS CRISTINA ANDRE...,3323,CFIBCCHIGA,327.00,971.00,BTG
...,...,...,...,...,...,...,...,...,...
957,2025-02,25880866-5,0,Rolin FOURCROY BAPTISTE LOUIS...,9500,CFISCCH,31.00,42925.00,BTG
958,2025-02,76104590-3,0,INVERSIONES Y ASESORIAS ALTOS DE SANTA FE LTDA...,3308,CFISCCH,4.00,42925.00,BTG
959,2025-02,77475773-2,0,SOCIEDAD DE PRESTACIONES MÉDICAS AGUSTINAS SPA...,9500,CFISCCH,102.00,42925.00,BTG
960,2025-02,77517228-2,0,SOCIEDAD MEDICA E INVERSIONES GALLEGOS MORAN S...,9500,CFISCCH,5.00,42925.00,BTG


In [8]:
Nemo=Facturacion['Nemo'].to_list()
Nemo
#Facturacion['Fact'].sum()


['CFISCCH',
 'CFIBTGRE-E',
 'CFIBPDCCHA',
 'CFIBTGPLAA',
 'CFIBTGCYFA',
 'CFIBTGRCA',
 'CFICELDI1',
 'CFIBTGFGIA',
 'CFIBTGRRH',
 'CFICELDI2',
 'CFIBTGRRA',
 'CFIBCCHIGA',
 'CFIBTGDEA',
 'CFICGSCH-B      ',
 'CFICGGTA-E']

In [9]:
#i='CFITACTI-A'
#patrimonio['cuotasNVS']=patrimonio.query('Nemo=="CFITACTI-A"')['CANTIDAD_CUOTAS'].sum()
#patrimonio['prorratra']=patrimonio['CANTIDAD_CUOTAS']*202789.075630252/patrimonio['cuotasNVS']
#print(patrimonio.query('Nemo=="CFISCCH"')) # si da cero es por problema de datos, losnemos tienen espacios

#patrimonio['cuotasNVS']=patrimonio.query('Nemo=="CFITACTI-A"')['CANTIDAD_CUOTAS'].sum()
#patrimonio

In [10]:
#No paga

No_pago_Sartor =[9500,7696,7697,7699]
No_pago_HMC =  [6800, 7000, 7200, 7201, 7600, 7696, 7697, 9500, 9600]  #[9500,7000, 7200,7201,9600]
No_pago_Credicorp = [9500, 7600, 7697, 7696]
No_pago_BCI =[9500]
No_pago_Toesca=[6800]
No_pago_MBI =[9500, 7600, 7697, 7696]
No_pago_BTG =[9500]
No_pago_Compass=[9500, 7600, 7697, 7696]



test =patrimonio
for index in range(test.shape[0]):
    if test.iloc[index,8]=='HMC' and test.iloc[index,4] in No_pago_HMC:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Sartor' and test.iloc[index,4] in No_pago_Sartor:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
# nuevo 2025-03
# no pago credicorp

for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Credicorp' and test.iloc[index,4] in No_pago_Credicorp:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
#No pago BCI
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='BCI' and test.iloc[index,4] in No_pago_BCI:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0

#No pago Toesca
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Toesca' and test.iloc[index,4] in No_pago_Toesca:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
#no pago MBI
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='MBI' and test.iloc[index,4] in No_pago_MBI:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
#no pago BTG

for index in range(test.shape[0]):        
    if test.iloc[index,8]=='BTG' and test.iloc[index,4] in No_pago_BTG:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Compass' and test.iloc[index,4] in No_pago_Compass:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0

In [11]:
test.query('Cliente=="No Pago"')

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
29,2025-02,14549417-6,2,No Pago,9500,CFIBTGCYFA,0.00,36292.00,BTG
30,2025-02,17754446-9,0,No Pago,9500,CFIBTGCYFA,0.00,36292.00,BTG
31,2025-02,19185569-8,0,No Pago,9500,CFIBTGCYFA,0.00,36292.00,BTG
37,2025-02,4427638-0,0,No Pago,9500,CFIBTGPLAA,0.00,381899.00,BTG
38,2025-02,5361458-2,1,No Pago,9500,CFIBTGPLAA,0.00,381899.00,BTG
...,...,...,...,...,...,...,...,...,...
956,2025-02,24732559-K,1,No Pago,9500,CFISCCH,0.00,42925.00,BTG
957,2025-02,25880866-5,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG
959,2025-02,77475773-2,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG
960,2025-02,77517228-2,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG


In [12]:

print(index)

961


In [13]:
test

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
0,2025-02,5199497-3,0,TAVERNE DENNISON ANABELLA EILY ...,7200,CFIBCCHIGA,102.00,971.00,BTG
1,2025-02,7586678-K,0,ESCOBAR HENRIQUEZ RAUL GUILLERMO...,3314,CFIBCCHIGA,129.00,971.00,BTG
2,2025-02,11347653-2,0,VELASCO PALMA FRANCISCA ...,7200,CFIBCCHIGA,930.00,971.00,BTG
3,2025-02,12752532-3,2,INVERNIZZI CASTILLO XIMENA BEATRIZ...,1638,CFIBCCHIGA,8297.00,971.00,BTG
4,2025-02,13990767-1,0,CABALLERO VEAS CRISTINA ANDRE...,3323,CFIBCCHIGA,327.00,971.00,BTG
...,...,...,...,...,...,...,...,...,...
957,2025-02,25880866-5,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG
958,2025-02,76104590-3,0,INVERSIONES Y ASESORIAS ALTOS DE SANTA FE LTDA...,3308,CFISCCH,4.00,42925.00,BTG
959,2025-02,77475773-2,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG
960,2025-02,77517228-2,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG


In [14]:

Nemo=Facturacion.query('Fact>0').copy()
Nemo=Nemo['Nemo'].to_list()
print(Nemo)
print(len(Nemo))


['CFISCCH', 'CFIBTGRE-E', 'CFIBPDCCHA', 'CFIBTGPLAA', 'CFIBTGCYFA', 'CFIBTGRCA', 'CFICELDI1', 'CFIBTGFGIA', 'CFIBTGRRH', 'CFICELDI2', 'CFIBTGRRA', 'CFIBCCHIGA', 'CFIBTGDEA', 'CFICGSCH-B      ', 'CFICGGTA-E']
15


In [15]:
for i in Nemo:
    if patrimonio.query('Nemo=="'+i+'"').empty :
        print(i)

CFIBTGRE-E
CFICGSCH-B      
CFICGGTA-E


In [16]:
inic=Nemo[0]
print(inic)
Carga=patrimonio.query('Nemo=="'+inic+'"').copy()

Carga['cuotasNVS']=Carga.query('Nemo=="'+inic+'"')['CANTIDAD_CUOTAS'].sum()
Carga['monto']=Carga['CANTIDAD_CUOTAS']
Carga['acciones']=Carga['CANTIDAD_CUOTAS']
print('1')
Carga['comision']=Carga['CANTIDAD_CUOTAS']*float(Facturacion.query('Nemo=="'+inic+'"')['Fact'])/Carga['cuotasNVS']
Carga['porcentaje']=Carga['CANTIDAD_CUOTAS']/Carga['cuotasNVS']
print('1')
Carga['emisor'] = Facturacion.query('Nemo=="'+inic+'"').iat[0,2]
Carga['mercado'] = 'RV'
Carga['FECHA'] = '30-07-2022'

CFISCCH
1
1


In [17]:
#Carga=patrimonio.query('Nemo=="CFISCCH"').copy()
Carga

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor,cuotasNVS,monto,acciones,comision,porcentaje,mercado,FECHA
912,2025-02,8427228-0,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
913,2025-02,9497244-2,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
914,2025-02,9913936-6,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
915,2025-02,9979324-4,1,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
916,2025-02,9984895-2,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
917,2025-02,12246880-1,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
918,2025-02,12562443-K,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
919,2025-02,12699269-6,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
920,2025-02,12856206-0,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
921,2025-02,12989521-7,2,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022


In [18]:
for i in Nemo[1:]:
    if patrimonio.query('Nemo=="'+i+'"').empty==False:
        dfpaso=patrimonio.query('Nemo=="'+i+'"').copy()
        dfpaso['cuotasNVS']=dfpaso.query('Nemo=="'+i+'"')['CANTIDAD_CUOTAS'].sum()
        dfpaso['monto']=dfpaso['CANTIDAD_CUOTAS']
        dfpaso['acciones']=dfpaso['CANTIDAD_CUOTAS']
        dfpaso['comision']=dfpaso['CANTIDAD_CUOTAS']*float(Facturacion.query('Nemo=="'+i+'"')['Fact'])/dfpaso['cuotasNVS']
        dfpaso['porcentaje']=dfpaso['CANTIDAD_CUOTAS']/dfpaso['cuotasNVS']
        dfpaso['emisor'] = Facturacion.query('Nemo=="'+i+'"').iat[0,2]
        dfpaso['mercado'] = 'RV'
        dfpaso['FECHA'] = '30-07-2022'
        Carga=pd.concat([Carga,dfpaso], sort=False)

###### Carga.keys()

In [19]:
Carga.reset_index()

,index,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor,cuotasNVS,monto,acciones,comision,porcentaje,mercado,FECHA
0,912,2025-02,8427228-0,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
1,913,2025-02,9497244-2,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
2,914,2025-02,9913936-6,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
3,915,2025-02,9979324-4,1,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
4,916,2025-02,9984895-2,0,No Pago,9500,CFISCCH,0.00,42925.00,BTG,15.00,0.00,0.00,0.00,0.00,RV,30-07-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,5,2025-02,19308867-8,0,LOPEZ ARNELLO DIEGO NICOLAS ...,7200,CFIBCCHIGA,35.00,971.00,BTG,10895.00,35.00,35.00,3.12,0.00,RV,30-07-2022
958,6,2025-02,19606029-4,0,TAVERNE VELASCO CRISTIAN ...,7200,CFIBCCHIGA,352.00,971.00,BTG,10895.00,352.00,352.00,31.37,0.03,RV,30-07-2022
959,7,2025-02,19893304-K,0,VERDUGO DAMM MARTÍN ...,3314,CFIBCCHIGA,187.00,971.00,BTG,10895.00,187.00,187.00,16.67,0.02,RV,30-07-2022
960,8,2025-02,76744616-0,0,MIBGROUP AVIATION LIMITADA ...,3333,CFIBCCHIGA,536.00,971.00,BTG,10895.00,536.00,536.00,47.77,0.05,RV,30-07-2022


In [20]:
CargaCSV=Carga[['Rut','monto','acciones','porcentaje','comision','emisor' ,'Nemo','mercado','FECHA']]

In [21]:
CargaCSV.to_excel('CargafipLAST0225.xlsx')

In [22]:
CargaCSV.info()


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rut         0 non-null      object 
 1   monto       0 non-null      float64
 2   acciones    0 non-null      float64
 3   porcentaje  0 non-null      float64
 4   comision    0 non-null      float64
 5   emisor      0 non-null      object 
 6   Nemo        0 non-null      object 
 7   mercado     0 non-null      object 
 8   FECHA       0 non-null      object 
dtypes: float64(4), object(5)
memory usage: 0.0+ bytes
